# Creating a Country-Nationality Crosswalk

I needed a modified country-nationality crosswalk for my project, so I adapted a list put together by `Dinu`, which can be found [here](https://github.com/Dinu/country-nationality-list/blob/master/countries.csv).

I made some initial changes to the raw `countries.csv` before using it to build my crosswalk:
* Added "Ivory Coast"
* Added "Burma"
* Added "Great Britain"
* Added "North Korea"
* Added "South Korea"
* Added "Laos" and "Lao"
* Added "Russia"
* Added "Republic of Congo" and "Democratic Republic of Congo"
* Added duplicates for all islands with commas in them (i.e., Bonaire)
* Added "Tajik"
* Added "Turkmenistani"
* Added "U.S."
* Added "DPRK"
* Added "ROK"
* Removed all parentheses parts and comma parts from country names
* Modified "United States of America" to "United States"
* Modified "Vatican City State" to "Vatican City"
* Modified "Taiwan" entry to solely "Taiwanese" (previously contained "Chinese")
* Modified "Macau" entry to solely "Macanese" (previously contained "Chinese")
* Removed entry for "United States Minor Outlying Islands," so "American" will only return "United States"
* Removed entry for "China" (because I want to match on all *other* countries, excluding China)

#### Exploding rows that have multiple entries under `nationality`

Entries are separated by either a comma or 'or'.

In [ ]:
import pandas as pd

countries = pd.read_csv('C:data/countries.csv')
countries.head()

In [ ]:
c = countries.copy()
print(c.shape) # (265, 5)
c[c.nationality.str.contains(' or ')]#.shape

8 records with commas, 35 records with 'or', one with both a comma and 'or'.

In [ ]:
nc = c[c.nationality.str.contains(',')]
nc.head()

In [ ]:
d = pd.DataFrame(nc.nationality.str.split(',').tolist(), index=nc.en_short_name).stack()
d = d.reset_index()[[0, 'en_short_name']]
d.columns = ['nationality','en_short_name']
d = pd.DataFrame(d.nationality.str.split(' or ').tolist(), index=d.en_short_name).stack().reset_index()[[0, 'en_short_name']]
d.columns = ['nationality','en_short_name']
d.head()

In [ ]:
nc2 = c[c.nationality.str.contains(' or ')]
nc2

In [ ]:
e = pd.DataFrame(nc2.nationality.str.split(' or ').tolist(), index=nc2.en_short_name).stack()
e = e.reset_index()[[0, 'en_short_name']]
e.columns = ['nationality','en_short_name']
e = e[:14]
e

#### Merging the two tables with "exploded" records back into the main country table

In [ ]:
c = countries.copy()
d # comma
e # or
c = pd.merge(c, d, how='left', on='en_short_name')
c.nationality_y = c.nationality_y.fillna(c.nationality_x)

c = pd.merge(c, e, how='left', on='en_short_name')
c.nationality = c.nationality.fillna(c.nationality_y)
c = c.drop(columns=['nationality_x','nationality_y'])
c = c.drop_duplicates()
#c = c.reset_index()
c.en_short_name = c.en_short_name.str.lower()
c.nationality = c.nationality.str.lower()
c.nationality = c.nationality.str.strip()
c = c.rename(columns={'en_short_name':'country'})
c.to_csv('C:data/countries_clean.csv', index=False)
c.head()